In [34]:
#  búsqueda de valores faltantes, valores atípicos/extremos u outliers y registros duplicados.
import pandas as pd
import numpy as np
import openpyxl

Para manipular el archivo Excel usando Python instalamos la dependencia e importamos la biblioteca arriba

In [35]:
pip install openpyxl 

In [44]:
# Leer todas las hojas del archivo Excel
internet_dataset = "Datasets\Internet.xlsx"
internet_file = pd.ExcelFile(internet_dataset)

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7224\2447630919.py:2: SyntaxWarning: invalid escape sequence '\I'
  internet_dataset = "Datasets\Internet.xlsx"


In [51]:
# Recorrer todas las hojas para obtener un resumen descriptivo
for hoja in internet_file.sheet_names:
    print(f"\n--- Resumen de la hoja: {hoja} ---")
    df = pd.read_excel(internet_file, sheet_name=hoja)
    
    # Información general
    print("Información del DataFrame:")
    print(df.info())
    
    # Resumen estadístico
    print("\nResumen estadístico:")
    print(df.describe(include='all'))

    # Revisar valores nulos
    print("\nValores nulos por columna:")
    print(df.isnull().sum())
    
    # Revisar los primeros 5 datos para tener una idea de la estructura
    print("\nPrimeras 5 filas:")
    print(df.head())


--- Resumen de la hoja: Acc_vel_loc_sinrangos ---
Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3065 entries, 0 to 3064
Columns: 125 entries, Provincia to 10000 Mbps
dtypes: float64(121), object(4)
memory usage: 2.9+ MB
None

Resumen estadístico:
           Provincia    Partido Localidad Link Indec         Otros  \
count           3065       3065      3065       3065   1030.000000   
unique            24        432      2778       2664           NaN   
top     BUENOS AIRES  San Justo     Otros  Sin Datos           NaN   
freq             717         56        70         74           NaN   
mean             NaN        NaN       NaN        NaN    318.614563   
std              NaN        NaN       NaN        NaN   2964.939611   
min              NaN        NaN       NaN        NaN    -15.000000   
25%              NaN        NaN       NaN        NaN      1.000000   
50%              NaN        NaN       NaN        NaN      6.000000   
75%              NaN 

 Los nulos en ' Acc_vel_loc_sinrangos ' podrían representar usuarios no suscriptos. Vamos a reemplazarlos por 0 para no perder ningun valor. 

In [54]:
# Cargar la hoja Acc_vel_loc_sinrangos
hoja_acc_vel = pd.read_excel(internet_file, sheet_name='Acc_vel_loc_sinrangos')

# Reemplazar nulos por 0 en todas las columnas
hoja_acc_vel.fillna(0, inplace=True)

Como había mbps de hasta 68405 y el máximo de mbps es 1000, se indujo que fueron errores de data entry y se los cambió a 1000 por ser el máximo

In [55]:
# Establecer un umbral máximo de 1000 Mbps y reemplazar valores fuera de rango
velocidad_columnas = [col for col in hoja_acc_vel.columns if 'Mbps' in col]
for col in velocidad_columnas:
    hoja_acc_vel[col] = np.where(hoja_acc_vel[col] > 1000, 1000, hoja_acc_vel[col])

In [57]:
# Verificar los resultados después de la limpieza
print("Estadísticas descriptivas después de limpiar nulos y valores fuera de rango:")
#print(hoja_acc_vel.describe())

# Revisar los primeros 5 registros para verificar cómo quedó la limpieza
print("\nPrimeras 5 filas después de la limpieza:")
print(hoja_acc_vel.head())

Estadísticas descriptivas después de limpiar nulos y valores fuera de rango:

Primeras 5 filas después de la limpieza:
      Provincia     Partido          Localidad Link Indec  Otros  0,256 Mbps  \
0  BUENOS AIRES  25 de Mayo         25 de Mayo    6854100    0.0         0.0   
1  BUENOS AIRES  25 de Mayo    Agustín Mosconi    6854010    0.0         0.0   
2  BUENOS AIRES  25 de Mayo          Del Valle    6854020    0.0         0.0   
3  BUENOS AIRES  25 de Mayo          Ernestina    6854030    0.0         0.0   
4  BUENOS AIRES  25 de Mayo  Gobernador Ugarte    6854040    0.0         0.0   

   0,5 Mbps  0,512 Mbps  0,75 Mbps  1 Mbps  ...  680 Mbps  700 Mbps  800 Mbps  \
0       2.0         0.0       19.0     0.0  ...       0.0       0.0       0.0   
1       0.0         0.0        0.0     0.0  ...       0.0       0.0       0.0   
2       1.0         0.0        0.0     0.0  ...       0.0       0.0       0.0   
3       0.0         0.0        0.0     0.0  ...       0.0       0.0       0.

In [58]:
# Calcular el porcentaje de valores nulos y ceros por columna
nulos_por_columna = hoja_acc_vel.isnull().sum() / len(hoja_acc_vel) * 100
ceros_por_columna = (hoja_acc_vel == 0).sum() / len(hoja_acc_vel) * 100

# Mostrar las columnas con más del 80% de valores nulos o ceros
print("Columnas con más del 80% de valores nulos:")
print(nulos_por_columna[nulos_por_columna > 80])

print("\nColumnas con más del 80% de valores ceros:")
print(ceros_por_columna[ceros_por_columna > 80])

Columnas con más del 80% de valores nulos:
Series([], dtype: float64)

Columnas con más del 80% de valores ceros:
0,256 Mbps    98.825449
0,512 Mbps    82.871126
0,75 Mbps     90.734095
1,5 Mbps      99.510604
2,2 Mbps      99.967374
                ...    
999 Mbps      99.445351
1000 Mbps     86.296900
1024 Mbps     99.934747
4000 Mbps     99.967374
10000 Mbps    99.934747
Length: 107, dtype: float64


In [62]:
# Seleccionar las columnas numéricas (evitando las columnas de texto como Provincia, Partido, Localidad, etc.)
columnas_velocidad = hoja_acc_vel.columns[5:]  # Asumiendo que las primeras 5 columnas son texto

# Agrupar por provincia y sumar las columnas de velocidades
suscriptores_por_provincia = hoja_acc_vel.groupby('Provincia')[columnas_velocidad].sum()

# Mostrar el resultado
suscriptores_por_provincia.head()

,"0,256 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,...,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,999 Mbps,1000 Mbps,1024 Mbps,4000 Mbps,10000 Mbps
Provincia,,,,,,,,,,,,,,,,,,,,,
BUENOS AIRES,18.0,1847.0,169.0,4611.0,16579.0,141.0,18363.0,0.0,21.0,36880.0,...,0.0,0.0,563.0,0.0,0.0,26.0,112203.0,0.0,0.0,0.0
CABA,31.0,451.0,34.0,1000.0,1000.0,0.0,249.0,0.0,0.0,1000.0,...,0.0,1.0,1.0,0.0,0.0,25.0,1000.0,0.0,0.0,0.0
CATAMARCA,0.0,34.0,37.0,0.0,176.0,0.0,115.0,0.0,0.0,1644.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0
CHACO,0.0,234.0,2.0,0.0,416.0,0.0,780.0,0.0,0.0,2925.0,...,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0
CHUBUT,8.0,69.0,34.0,836.0,247.0,0.0,1870.0,0.0,0.0,4507.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [73]:
# Agrupar por Provincia, Partido y Localidad, y sumar las columnas de velocidades
suscriptores_por_localidad = hoja_acc_vel.groupby(['Provincia', 'Partido', 'Localidad'])[columnas_velocidad].sum().reset_index()

# Mostrar las primeras filas del resultado
suscriptores_por_localidad.head()

,Provincia,Partido,Localidad,"0,256 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,5 Mbps",2 Mbps,...,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,999 Mbps,1000 Mbps,1024 Mbps,4000 Mbps,10000 Mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,0.0,2.0,0.0,19.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BUENOS AIRES,25 de Mayo,Del Valle,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BUENOS AIRES,25 de Mayo,Ernestina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,0.0,0.0,0.0,0.0,0.0,0.0,106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# Agrupar por Provincia y Partido, y sumar las columnas de velocidades
suscriptores_por_partido = hoja_acc_vel.groupby(['Provincia', 'Partido'])[columnas_velocidad].sum().reset_index()
suscriptores_por_partido.head()

,Provincia,Partido,"0,256 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,5 Mbps",2 Mbps,"2,2 Mbps",...,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,999 Mbps,1000 Mbps,1024 Mbps,4000 Mbps,10000 Mbps
0,BUENOS AIRES,25 de Mayo,0.0,3.0,0.0,19.0,0.0,0.0,280.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BUENOS AIRES,9 de Julio,0.0,0.0,0.0,9.0,9.0,0.0,193.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
2,BUENOS AIRES,AMBA,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BUENOS AIRES,Adolfo Alsina,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BUENOS AIRES,Adolfo Gonzales Chaves,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
